# 决策树

决策树的主要优势就在于数据形式非常容易理解，而这也是kNN最主要的缺点之一；

决策树：
	1. 根据某一特征进行数据集划分
	2. 判断划分后各个子数据集中的数据是否均为同一类型的
		是：返回节点类型
		否：使用当前子数据集创建新节点，返回步骤1处理新节点
	关键：
		1. 何时停止划分数据集，也是递归结束的条件ID3（ID3处理如何划分数据集，以及何时停止）；
		2. 如何决定使用那个特征划分数据集，根据香农熵，选择最大信息增益的特征划分；

In [1]:
import numpy as np
import math

## 决策树的构造

### 信息增益

划分数据集的大原则是:将无序的数据变得更加有序，方法有很多也各有优劣，其中一种方法是使用信息论度量信息；

划分数据集前后信息的变化称之为信息增益，知道如何计算信息增益，我们就可以获取到使得信息增益最大的那个特征作为划分数据集的特征，此处需要使用香农熵来计算信息增益；

#### 计算香农熵

    熵定义为信息的期望值，在明晰这个概念之前，我们必须知道信息的定义，如果待分类的事务可能划分在多个分类之中，则符号 x_i 的信息定义为：
	l(x_i) = -log_2 p(x_i)
	其中p(x_i)是选择该分类的概率；
	为了计算熵，我们需要计算所有类别所有可能值包含的信息期望值，通过下面的公式得到：
	-sum(p(x_i)*log_2 p(x_i)) i={1,2,3.....n} n为目标变量总类别数
	从公式可以看出含义是：各个类别自身的概率乘以自己的信息量，最后求和得到熵；
    
    可以说香农熵就是用于度量数据分类的无序程度，因此分类越多，熵会越大；

#### 实现

In [44]:
def calcShannonEntropy(dataSet):
		"""
		统计类别信息
		1. 获取到所有分类，创建字典
		2. 为每个字典设置其对应分类在数据集中数量
		3. 通过数量和总数计算熵
		"""
		classes = {}
		for d in dataSet:
			classes[d[-1]] = classes.get(d[-1],0)+1 # 如果没有则设置-1+1，即0，如果有则加1
		count = 1.*len(dataSet)
		entropy = -sum([classes[k]/count*math.log(classes[k]/count,2) for k in classes.keys()])
		return entropy

calcShannonEntropy(np.array([[1,1,'A'],[1,2,'B'],[2,1,'A'],[2,3,'A'],[3,2,'B']]))

0.9709505944546686

In [8]:
calcShannonEntropy(np.array([[1,1,'A'],[1,2,'B'],[2,1,'A'],[2,3,'A'],[3,2,'B'],[3,2,'A'],[3,2,'F'],[3,2,'E'],[3,2,'E'],[3,2,'D']]))

{'A': 4, 'B': 2, 'E': 2, 'D': 1, 'F': 1}
2.12192809489


2.1219280948873624

熵越高,则混合的数据也越多；

### 划分数据集

In [70]:
def splitDataSet(dataSet, axis, value): # 改善一下，原代码是根据value，每次返回当前数据集中对应特征值为value的数据子集，改为返回所有value的数据子集，以字典形式返回
	"""
	将数据集根据某个特征axis分割为每个特征值对应的数据子集，以dict形式返回
	"""
	dataDict = {}
	for d in dataSet:
		v = dataDict.get(d[axis],[]) # 从列表中创建集合是Python语言得到列表中唯一元素值的最快方法
		v.append(list(np.append(d[:axis],d[axis+1:])))
		dataDict[d[axis]] = v
	return dataDict

In [71]:
splitDataSet(np.array([[1,1,'A'],[1,2,'B'],[2,1,'A'],[2,3,'A'],[3,2,'B']]), 0, None)

{'1': [['1', 'A'], ['2', 'B']],
 '2': [['1', 'A'], ['3', 'A']],
 '3': [['2', 'B']]}

### 选择最佳特征

In [72]:
def chooseBestFeature2Split(dataSet):
	"""
	选择将数据集划分后总熵最小的那个特征以及对应的熵值，数据子集的dict形式返回
	"""
	axiss = len(dataSet[0])-1 # 去掉最后一个目标变量，否则肯定是该变量最符合要求，按照目标变量划分数据再用目标变量来评估xD
	minEntropy = 99999999 # 最初的无序度量值,用于与划分完之后的数据集计算的熵值进行比较
	minAxis = -1
	minDataDict = None
	for axis in range(axiss):
		dataDict = splitDataSet(dataSet, axis, None)
		sumEntropy = sum([calcShannonEntropy(dataDict[k]) for k in dataDict.keys()])
		if sumEntropy < minEntropy:
			minEntropy = sumEntropy
			minAxis = axis
			minDataDict = dataDict
	print 'min entropy:'+str(minEntropy)
	print 'min axis:'+str(minAxis)
	return minEntropy, minAxis, minDataDict

In [73]:
chooseBestFeature2Split(np.array([[1,1,'A'],[1,2,'B'],[2,1,'A'],[2,3,'A'],[3,2,'B']]))

min entropy:0.0
min axis:1


(0.0,
 1,
 {'1': [['1', 'A'], ['2', 'A']],
  '2': [['1', 'B'], ['3', 'B']],
  '3': [['2', 'A']]})

### 递归构建决策树

递归结束的条件是:程序**遍历完**所有划分数据集的属性,或者每个分支下的所有实例都具有
**相同**的分类。如果所有实例具有相同的分类,则得到一个叶子节点或者终止块。任何到达叶子节
点的数据必然属于叶子节点的分类。

如果数据集已经处理了所有属性,但是类标签依然不是唯一
的,此时我们需要决定如何定义该叶子节点,在这种情况下,我们通常会采用**多数表决**的方法决
定该叶子节点的分类。

In [74]:
def majorityCnt(classList):
	"""
	计算最多的分类并返回，用于处理在所有特征都使用完之后数据子集中依然存在多个分类的case
	"""
	classDict = {}
	for c in classList:
		classDict[c] = classDict.get(c,0)+1
	return sorted(classDict.items(),key = lambda x:x[1],reverse = True)[0][0]

majorityCnt(['a','b','b','a','a'])

'a'

In [75]:
def createTree(dataSet, labels):
    """
    递归构建决策树
    结束条件：
        1. 当前数据集均为同一分类，此时熵为0；
        2. 当前数据集已经没有特征可以用于划分；
    返回：
        1. 返回分类标签
    """
    if calcShannonEntropy(dataSet) == 0:
        print '数据集在同一分类：'+str(labels[0])
        return labels[0]
    elif len(dataSet[0])==1:
        print '投票表决：'+str(majorityCnt(labels))
        return majorityCnt(labels)
    else:
        _,_,dataDict = chooseBestFeature2Split(dataSet)
        for k in dataDict.keys():
            createTree(dataDict[k], [i[-1] for i in dataDict[k]])

createTree(np.array([[1,1,'B'],[2,1,'B'],[2,1,'A'],[2,3,'A'],[3,2,'B']]), ['A','B','A','A','B'])

min entropy:0.918295834054
min axis:0
数据集在同一分类：B
数据集在同一分类：B
min entropy:1.0
min axis:0
投票表决：A
数据集在同一分类：A


In [80]:
def createTree(dataSet, labels):
	"""
	递归构建决策树
	结束条件：
		1. 当前数据集均为同一分类，此时熵为0；
		2. 当前数据集已经没有特征可以用于划分；
	返回：
		1. 返回分类标签
	"""
	if calcShannonEntropy(dataSet) == 0:
		return '数据集在同一分类：'+str(dataSet[0][-1])
		#return dataSet[0][-1]
	elif len(dataSet[0])==1:
		return '投票表决：'+str(majorityCnt(list(np.array(dataSet)[:,-1])))
		#return majorityCnt(list(np.array(dataSet)[:,-1]))
	
	_,bestFeat,dataDict = chooseBestFeature2Split(dataSet)
	bestFeatLabel = labels[bestFeat]
	del labels[bestFeat]
	myTree = {bestFeatLabel:{}}
	for k in dataDict.keys():
		tmpLabels = labels[:]
		myTree[bestFeatLabel][k] = createTree(dataDict[k], tmpLabels)#[i[-1] for i in dataDict[k]])
	return myTree
			
myTree = createTree(np.array([[1,1,'B'],[2,1,'B'],[2,1,'A'],[2,3,'A'],[3,2,'B']]), ['feature 1','feature 2','target'])
print myTree

min entropy:0.918295834054
min axis:0
min entropy:1.0
min axis:0
{'feature 1': {'1': '\xe6\x95\xb0\xe6\x8d\xae\xe9\x9b\x86\xe5\x9c\xa8\xe5\x90\x8c\xe4\xb8\x80\xe5\x88\x86\xe7\xb1\xbb\xef\xbc\x9aB', '3': '\xe6\x95\xb0\xe6\x8d\xae\xe9\x9b\x86\xe5\x9c\xa8\xe5\x90\x8c\xe4\xb8\x80\xe5\x88\x86\xe7\xb1\xbb\xef\xbc\x9aB', '2': {'feature 2': {'1': '\xe6\x8a\x95\xe7\xa5\xa8\xe8\xa1\xa8\xe5\x86\xb3\xef\xbc\x9aA', '3': '\xe6\x95\xb0\xe6\x8d\xae\xe9\x9b\x86\xe5\x9c\xa8\xe5\x90\x8c\xe4\xb8\x80\xe5\x88\x86\xe7\xb1\xbb\xef\xbc\x9aA'}}}}
